In [3]:
import warnings
warnings.filterwarnings(action='ignore')
%config Computer.use_jedi = False
import numpy as np
import pandas as pd
import requests

In [4]:
subway = pd.read_csv('./data/지하철위도경도(신).csv')
subway

,외구간_역_수,역한글명칭,호선명칭,환승역X좌표,환승역Y좌표
0,4128,삼성중앙,9호선(연장),127.053282,37.513011
1,4124,사평,9호선,127.015259,37.504206
2,4121,구반포,9호선,126.987332,37.501364
3,4119,흑석(중앙대입구),9호선,126.963708,37.508770
4,4116,샛강,9호선,126.928422,37.517274
...,...,...,...,...,...
760,4411,관악산(서울대),신림선,126.945064,37.469102
761,4306,신논현,신분당선(연장2),127.025060,37.504598
762,1404,탕정,장항선,127.084850,36.788660
763,4304,신사,신분당선(연장2),127.020114,37.516334


In [5]:
# 네이버 개발자 포털에서 발급한 Client ID와 Client Secret
client_id = '5j924usqjd'
client_secret = 'W4wjDI2jRzuvP1N0zo9UiG1zTRoDP30idjpESbp4'

# '위도'와 '경도' 정보가 있는 데이터프레임 (house_df)에서 반복적으로 역 지오코딩을 수행합니다.
for index, row in subway.iterrows():
    latitude = row['환승역Y좌표']
    longitude = row['환승역X좌표']
    if latitude is not None and longitude is not None:
        url = f'https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords={longitude},{latitude}&sourcecrs=epsg:4326&output=json&orders=addr,admcode'
        headers = {
            'X-NCP-APIGW-API-KEY-ID': client_id,
            'X-NCP-APIGW-API-KEY': client_secret
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if data.get('results'):
                gu = data['results'][0]['region']['area2']['name']  # 예시로 주소 중 'area3'의 'name' 사용
                dong = data['results'][0]['region']['area3']['name']  # 예시로 주소 중 'area3'의 'name' 사용
                subway.at[index, '법정동'] = dong  # 주소 정보를 데이터프레임에 추가
                subway.at[index, '시군구'] = gu  # 주소 정보를 데이터프레임에 추가
                
subway

,외구간_역_수,역한글명칭,호선명칭,환승역X좌표,환승역Y좌표,법정동,시군구
0,4128,삼성중앙,9호선(연장),127.053282,37.513011,삼성동,강남구
1,4124,사평,9호선,127.015259,37.504206,반포동,서초구
2,4121,구반포,9호선,126.987332,37.501364,반포동,서초구
3,4119,흑석(중앙대입구),9호선,126.963708,37.508770,흑석동,동작구
4,4116,샛강,9호선,126.928422,37.517274,여의도동,영등포구
...,...,...,...,...,...,...,...
760,4411,관악산(서울대),신림선,126.945064,37.469102,신림동,관악구
761,4306,신논현,신분당선(연장2),127.025060,37.504598,역삼동,강남구
762,1404,탕정,장항선,127.084850,36.788660,탕정면,아산시
763,4304,신사,신분당선(연장2),127.020114,37.516334,논현동,강남구


In [6]:
subway.drop(columns=['외구간_역_수', '호선명칭', '환승역X좌표', '환승역Y좌표'], inplace=True)
subway

,역한글명칭,법정동,시군구
0,삼성중앙,삼성동,강남구
1,사평,반포동,서초구
2,구반포,반포동,서초구
3,흑석(중앙대입구),흑석동,동작구
4,샛강,여의도동,영등포구
...,...,...,...
760,관악산(서울대),신림동,관악구
761,신논현,역삼동,강남구
762,탕정,탕정면,아산시
763,신사,논현동,강남구


In [7]:
subway_dongbuk = subway[(subway['시군구'] == '강북구') | (subway['시군구'] == '도봉구') | (subway['시군구'] == '노원구')| (subway['시군구'] == '중랑구')
                            | (subway['시군구'] == '성북구')| (subway['시군구'] == '동대문구')]
subway_dongbuk

,역한글명칭,법정동,시군구
10,청량리(서울시립대입구),전농동,동대문구
15,한성대입구(삼선교),삼선동1가,성북구
20,수유(강북구청),번동,강북구
26,제기동,용두동,동대문구
47,망우,상봉동,중랑구
...,...,...,...
667,용두(동대문구청),용두동,동대문구
670,당고개,상계동,노원구
680,도봉,도봉동,도봉구
681,회기,휘경동,동대문구


In [8]:
# 숫자 + '가'가 들어가는 법정동을 '시가지동'으로 변경
subway_dongbuk['법정동'] = subway_dongbuk['법정동'].apply(lambda x: '시가지동' if x.endswith('가')  or x == '돈암동' else x)

# 변경된 데이터프레임 출력
subway_dongbuk

,역한글명칭,법정동,시군구
10,청량리(서울시립대입구),전농동,동대문구
15,한성대입구(삼선교),시가지동,성북구
20,수유(강북구청),번동,강북구
26,제기동,용두동,동대문구
47,망우,상봉동,중랑구
...,...,...,...
667,용두(동대문구청),용두동,동대문구
670,당고개,상계동,노원구
680,도봉,도봉동,도봉구
681,회기,휘경동,동대문구


In [9]:
subway_dongbuk.to_csv('./data/지하철주소동북.csv')
subway_dongbuk

,역한글명칭,법정동,시군구
10,청량리(서울시립대입구),전농동,동대문구
15,한성대입구(삼선교),시가지동,성북구
20,수유(강북구청),번동,강북구
26,제기동,용두동,동대문구
47,망우,상봉동,중랑구
...,...,...,...
667,용두(동대문구청),용두동,동대문구
670,당고개,상계동,노원구
680,도봉,도봉동,도봉구
681,회기,휘경동,동대문구


In [10]:
# '시군구명'과 '법정동명'을 기준으로 그룹화하고 '법정동'의 데이터 수를 카운트
grouped = subway_dongbuk.groupby(['시군구', '법정동']).size().reset_index(name='역수')
grouped

,시군구,법정동,역수
0,강북구,미아동,5
1,강북구,번동,1
2,강북구,수유동,2
3,강북구,우이동,3
4,노원구,공릉동,4
5,노원구,상계동,6
6,노원구,월계동,2
7,노원구,중계동,1
8,노원구,하계동,1
9,도봉구,도봉동,3


In [11]:
grouped.to_csv('./data/동북부지하철수.csv')
grouped

,시군구,법정동,역수
0,강북구,미아동,5
1,강북구,번동,1
2,강북구,수유동,2
3,강북구,우이동,3
4,노원구,공릉동,4
5,노원구,상계동,6
6,노원구,월계동,2
7,노원구,중계동,1
8,노원구,하계동,1
9,도봉구,도봉동,3


In [12]:
width = pd.read_csv('./data/동북부면적.csv')
width = width.drop(columns=['Unnamed: 0'], axis=1)
width

,시군구,법정동,면적
0,동대문구,신설동,0.22
1,동대문구,용두동,0.76
2,동대문구,제기동,0.72
3,동대문구,전농동,0.94
4,동대문구,답십리동,1.02
5,동대문구,장안동,1.47
6,동대문구,청량리동,0.62
7,동대문구,회기동,0.28
8,동대문구,휘경동,0.86
9,동대문구,이문동,0.91


In [13]:
width = pd.merge(width, grouped, on=['시군구', '법정동'], how='left')
width

,시군구,법정동,면적,역수
0,동대문구,신설동,0.22,2.0
1,동대문구,용두동,0.76,2.0
2,동대문구,제기동,0.72,NaN
3,동대문구,전농동,0.94,2.0
4,동대문구,답십리동,1.02,NaN
5,동대문구,장안동,1.47,NaN
6,동대문구,청량리동,0.62,NaN
7,동대문구,회기동,0.28,NaN
8,동대문구,휘경동,0.86,1.0
9,동대문구,이문동,0.91,2.0


In [14]:
width['역수'].fillna(0, inplace=True)
width

,시군구,법정동,면적,역수
0,동대문구,신설동,0.22,2.0
1,동대문구,용두동,0.76,2.0
2,동대문구,제기동,0.72,0.0
3,동대문구,전농동,0.94,2.0
4,동대문구,답십리동,1.02,0.0
5,동대문구,장안동,1.47,0.0
6,동대문구,청량리동,0.62,0.0
7,동대문구,회기동,0.28,0.0
8,동대문구,휘경동,0.86,1.0
9,동대문구,이문동,0.91,2.0


In [15]:
width['면적(㎢)당 역 수'] = width['역수'] / width['면적'] * 0.001
width

,시군구,법정동,면적,역수,면적(㎢)당 역 수
0,동대문구,신설동,0.22,2.0,0.009091
1,동대문구,용두동,0.76,2.0,0.002632
2,동대문구,제기동,0.72,0.0,0.000000
3,동대문구,전농동,0.94,2.0,0.002128
4,동대문구,답십리동,1.02,0.0,0.000000
5,동대문구,장안동,1.47,0.0,0.000000
6,동대문구,청량리동,0.62,0.0,0.000000
7,동대문구,회기동,0.28,0.0,0.000000
8,동대문구,휘경동,0.86,1.0,0.001163
9,동대문구,이문동,0.91,2.0,0.002198


In [16]:
width.drop(columns=['면적', '역수'], inplace=True)
width

,시군구,법정동,면적(㎢)당 역 수
0,동대문구,신설동,0.009091
1,동대문구,용두동,0.002632
2,동대문구,제기동,0.000000
3,동대문구,전농동,0.002128
4,동대문구,답십리동,0.000000
5,동대문구,장안동,0.000000
6,동대문구,청량리동,0.000000
7,동대문구,회기동,0.000000
8,동대문구,휘경동,0.001163
9,동대문구,이문동,0.002198


In [17]:
width.to_csv('./data/동북부지하철수.csv')
width

,시군구,법정동,면적(㎢)당 역 수
0,동대문구,신설동,0.009091
1,동대문구,용두동,0.002632
2,동대문구,제기동,0.000000
3,동대문구,전농동,0.002128
4,동대문구,답십리동,0.000000
5,동대문구,장안동,0.000000
6,동대문구,청량리동,0.000000
7,동대문구,회기동,0.000000
8,동대문구,휘경동,0.001163
9,동대문구,이문동,0.002198
